In [2]:
import numpy as np
import pandas as pd
from DW import *

class TSP(object):
    #城市坐标数组
    citys = np.array([])
    #城市名数组
    citys_name = np.array([])
    #种群大小
    pop_size = 50
    #交叉率
    c_rate = 0.7
    #突变率
    m_rate = 0.05
    #种群数组
    pop = np.array([])
    #适应度数组
    fitness = np.array([])
    #标记城市数目
    city_size = -1
    #最大迭代次数
    ga_num = 200
    #记录目前最优距离
    best_dist = -1
    #记录目前最优旅行方案
    best_gen = []
    #绘图类
    dw = Draw()
    
    def __init__(self, c_rate, m_rate, pop_size, ga_num):
        self.fitness = np.zeros(self.pop_size)
        self.c_rate = c_rate
        self.m_rate = m_rate
        self.pop_size = pop_size
        self.ga_num = ga_num
        
    def init(self):
        tsp = self
        #加载城市数据
        tsp.load_citys()
        #创建种群
        tsp.pop = tsp.create_pop(tsp.pop_size)
        #计算初始种群适应度
        tsp.fitness = tsp.get_fitness(tsp.pop)
        #计算绘图时的X界
        tsp.dw.bound_x = [np.min(tsp.citys[:, 0]), np.max(tsp.citys[:, 0])]
        #计算绘图时的Y界
        tsp.dw.bound_y = [np.min(tsp.citys[:, 1]), np.max(tsp.citys[:, 1])]
        #设置边界
        tsp.dw.set_xybound(tsp.dw.bound_x, tsp.dw.bound_y)
    
    #加载城市数据
    def load_citys(self):
        data = pd.read_csv(file = 'china.csv', delimiter = ';', header = None).values
        self.citys = data[:, 1:]
        self.citys_name = data[:, 0]
        self.city_size = data.shape[0]
    
    #生成种群
    def create_pop(self, size):
        pop = []
        for i in range(size):
            gene = np.arange(self.citys.shape[0])
            #打乱数组
            np.random.shuffle(gene)
            pop.append(gene)
        return np.array(pop)
    
    #主程序：迭代进化种群
    def evolution(self):
        tsp = self
        for i in range(self.ga_num):
            best_f_index = np.argmax(tsp.fitness)
            worst_f_index = np.argmin(tsp.fitness)
            local_best_gen = tsp.pop[best_f_index]
            local_best_dist = tsp.gen_distance(local_best_gen)
            
            if i == 0:
                tsp.best_gen = local_best_gen
                tsp.best_dist = local_best_dist
                
            if local_best_dist < tsp.best_dist:
                tsp.best_dist = local_best_dist
                tsp.best_gen = local_best_gen
                
                tsp.dw.ax.cla()
                tsp.re_draw()
                tsp.dw.plt.pause(0.001)
            else:
                tsp.pop[worst_f_index] = self.best_gen
            print('gen:%d evo,best dist :%s' % (i, self.best_dist))
            
            tsp.pop = tsp.select_pop(tsp.pop)
            tsp.fitness = tsp.get_fitness(tsp.pop)
            
            for j in range(self.pop_size):
                r = np.random.randint(0, self.pop_size - 1)
                if j != r:
                    tsp.pop[j] = tsp.cross(tsp.pop[j], tsp.pop[r])
                    tsp.pop[j] = tsp.mutate(tsp.pop[j])
            self.best_gen = self.EO(self.best_gen)
            tsp.best_dist = tsp.gen_distance(self.best_gen)
    
    #极值优化，传统遗传算法性能不好，这里混合EO
    #其会在整个基因的领域内，寻找一个最佳变换以更新基因
    def EO(self, gen):
        local_fitness = []
        for g in range(self.city_size):
            f = self.get_local_fitness(gen, g)
            local_fitness.append(f)
        max_city_i = np.argmax(local_fitness)
        maxgen = np.copy(gen)
        
        if 1 < max_city_i < self.city_size - 1:
            for j in range(max_city_i):
                maxgen = np.copy(gen)
                jj = max_city_i
                while jj < self.city_size:
                    gen1 = self.exchange(maxgen, j, jj)
                    d = self.gen_distance(maxgen)
                    d1 = self.gen_distance(gen1)
                    if d > d1:
                        maxgen = gen1[:]
                    jj += 1
        gen = maxgen
        return gen
    
    #选择种群，优胜劣汰，策略1：低于平均的要替换改变
    def select_pop(self, pop):
        best_f_index = np.argmax(self.fitness)
        av = np.median(self.fitness, axis = 0)
        for i in range(self.pop_size):
            if i != best_f_index and self.fitness[i] < av:
                pi = self.cross(pop[best_f_index], pop[i])
                pi = self.mutate(pi)
                pop[i, :] = pi[:]
        return pop
    
    #选择种群，优胜劣汰，策略2：轮盘赌，适应度低的替换的概率大
    def select_pop2(self, pop):
        probility = self.fitness / self.fitness.sum()
        idx = np.random.choice(np.arange(self.pop_size), size = self.pop_size, replace = True, p = probility)
        n_pop = pop[idx, :]
        return n_pop
    
    #交叉操作
    def cross(self, parent1, parent2):
        #交叉p1,p2的部分基因片段,多点交叉
        if np.random.rand() > self.c_rate:
            return parent1
        index1 = np.random.randint(0, self.city_size - 1)
        index2 = np.random.randint(index1, self.city_size - 1)
        # 交叉的基因片段
        tempGene = parent2[index1 : index2]
        
        newGene = []
        p1len = 0
        for g in parent1:
            if p1len == index1:
                # 插入基因片段
                newGene.extend(tempGene)
            if g not in tempGene:
                newGene.append(g)
            p1len += 1
        newGene = np.array(newGene)
        
        #交叉出错后重新生成一个
        if newGene.shape[0] != self.city_size:
            print('cross error')
            return self.create_pop(1)
        return newGene
    
    #基因突变
    def mutate(self, gene):
        if np.random.rand() > self.m_rate:
            return gene
        index1 = np.random.randint(0, self.city_size - 1)
        index2 = np.random.randint(index1, self.city_size - 1)
        newGene = self.reverse_gen(gene, index1, index2)
        
        #突变出错后重新生成一个
        if newGene.shape[0] != self.city_size:
            print('mutation error')
            return self.create_pop(1)
        return newGene
    
    ##函数：翻转基因中i到j之间的基因片段
    def reverse_gen(self, gen, i, j):
        if i >= j:
            return gen
        if j > self.city_size - 1:
            return gen
        parent1 = np.copy(gen)
        tempGene = parent1[i:j]
        newGene = []
        p1len = 0
        for g in parent1:
            if p1len == i:
                # 插入基因片段
                newGene.extend(tempGene[::-1])
            if g not in tempGene:
                newGene.append(g)
            p1len += 1
        return np.array(newGene)
    
    #函数：交换基因中i,j值
    def exchange_gen(self, gen, i, j):
        c = gen[j]
        gen[j] = gen[i]
        gen[i] = c
        return gen
    
    def get_fitness(self, pop):
        #适应度记录数组
        d = np.array([])
        for i in range(pop.shape[0]):
            # 取一条基因（编码解，个体）
            gen = pop[i]
             #计算此基因优劣（距离长短）
            dis = self.gen_distance(gen)
            #当前最优距离除以当前pop[i]（个体）距离；越近适应度越高，最优适应度为1
            dis = self.best_dist / dis
            # 保存适应度pop[i]
            d.append(dis)
        return d
    
    def get_local_fitness(self, gen, i):
        '''
        计算地i个城市的邻域
        交换基因数组中任意两个值组成的解集：称为邻域。计算领域内所有可能的适应度
        :param gen:城市路径
        :param i:第i城市
        :return:第i城市的局部适应度
        '''
        di = 0
        fi = 0
        if i == 0:
            di = self.ct_distance(self.citys[gen[0]], self.citys[gen[-1]])
        else:
            di = self.ct_distance(self.citys[gen[i]], self.citys[gen[i - 1]])
        od = []
        for j in range(self.city_size):
            if i != j:
                od.append(self.ct_distance(self.citys[gen[i]], self.citys[gen[i - 1]]))
        mind = np.min(od)
        fi = di - mind
        return fi
    
    #计算基因所代表的总旅行距离
    def gen_distance(self, gen):
        distance = 0.0
        for i in range(-1, len(self.citys) - 1):
            index1, index2 = gen[i], gen[i + 1]
            city1, city2 = self.citys[index1], self.citys[index2]
            distance += np.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)
        return distance

     #计算2城市之间的欧氏距离
    def ct_distance(self, city1, city2):
        d = np.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1] ** 2))
        return d

    #根据一条基因gen绘制一条旅行路线
    def draw_citys_way(self, gen):
        tsp = self
        dw = self.dw
        m = gen.shape[0]
        tsp.dw.set_xybound(tsp.dw.bound_x, tsp.dw.bound_y)
        for i in range(m):
            if i < m - 1:
                best_i = tsp.best_gen[i]
                next_best_i = tsp.best_gen[i + 1]
                best_icity = tsp.citys[best_i]
                next_best_icity = tsp.citys[next_best_i]
                dw.draw_line(best_icity, next_best_icity)
        start = tsp.citys[tsp.best_gen[0]]
        end = tsp.citys[tsp.best_gen[-1]]
        dw.draw_line(end, start)
    
    #根据一条基因gen绘制对应城市名称
    def draw_citys_name(self, gen, size = 5):
        tsp = self
        m = gen.shape[0]
        tsp.dw.set_xybound(tsp.dw.bound_x, tsp.dw.bound_y)
        for i in range(m):
            c = gen[i]
            best_icity = tsp.citys[c]
            tsp.dw.draw_text(best_icity[0], best_icity[1], tsp.citys_name[c], 10)
            
    #重绘图；每次迭代后绘制一次，动态展示。
    def re_draw(self):
        tsp = self
        tsp.dw.draw_points(tsp.citys[:, 0], tsp.citys[:, 1])
        tsp.draw_citys_name(tsp.pop[0], 8)
        tsp.draw_citys_way(self.best_gen)
    
def main():
    tsp = TSP(0.5, 0.1, 100, 500)
    tsp.init()
    tsp.evoluation()
    tsp.re_draw()
    tsp.dw.plt.show()

if __name__ == '__main__':
    main()

TypeError: parser_f() got an unexpected keyword argument 'file'

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei
